In [1]:
import pandas as pd
import numpy as np
import talib as ta
import os
import matplotlib.pyplot as plt
import warnings
import pymongo
from datetime import datetime
import pandas as pd
import numpy as np
from scipy import stats, signal
import plotly.express as px
import plotly.graph_objects as go

warnings.filterwarnings('ignore')

symbol = 'eth_usdt.spot:binance'
client = pymongo.MongoClient('172.16.20.81', 27017)
collection = client['Kline_1Min_Auto_Db_Plus'][symbol]
startTime = datetime(2022,1,1)
endTime = datetime(2022,12,31)
data_df = pd.DataFrame(
    list(collection.find({"datetime": {'$gt': startTime, '$lt': endTime}}))
)
data = data_df[['datetime', 'open', 'high', 'low', 'close', 'volume']].copy()
data = data.set_index('datetime')

In [27]:
kde_factor = 0.05
num_samples = 50
kde = stats.gaussian_kde(data.close.values,weights=data.volume.values,bw_method=kde_factor)
xr = np.linspace(data.close.min(),data.close.max(),num_samples)
kdy = kde(xr)
ticks_per_sample = (xr.max() - xr.min()) / num_samples

In [28]:
def get_dist_plot(c, v, kx, ky):
    fig = go.Figure()
    fig.add_trace(go.Histogram(name='Vol Profile', x=c, y=v, nbinsx=150, 
                               histfunc='sum', histnorm='probability density',
                               marker_color='#B0C4DE'))
    fig.add_trace(go.Scatter(name='KDE', x=kx, y=ky, mode='lines', marker_color='#D2691E'))
    return fig

get_dist_plot(data.close, data.volume, xr, kdy).show()

In [29]:
peaks,_ = signal.find_peaks(kdy)
pkx = xr[peaks]
pky = kdy[peaks]

pk_marker_args=dict(size=10)
fig = get_dist_plot(data.close, data.volume, xr, kdy)
fig.add_trace(go.Scatter(name="Peaks", x=pkx, y=pky, mode='markers', marker=pk_marker_args))

In [32]:
min_prom = kdy.max() * 0.75

width_range= 1
peaks, peak_props = signal.find_peaks(kdy, prominence=min_prom, width=width_range, rel_height=0.75)
pkx = xr[peaks]
pky = kdy[peaks]

In [33]:
left_ips = peak_props['left_ips']
right_ips = peak_props['right_ips']
width_x0 = xr.min() + (left_ips * ticks_per_sample)
width_x1 = xr.min() + (right_ips * ticks_per_sample)
width_y = peak_props['width_heights']

fig = get_dist_plot(data.close, data.volume, xr, kdy)
fig.add_trace(go.Scatter(name='Peaks', x=pkx, y=pky, mode='markers', marker=pk_marker_args))

for x0, x1, y in zip(width_x0, width_x1, width_y):
    fig.add_shape(type='line',
        xref='x', yref='y',
        x0=x0, y0=y, x1=x1, y1=y,
        line=dict(
            color='red',
            width=2,
        )
    )
fig.show()

In [9]:
pkx

array([1592.26])

In [8]:
width_x0

array([1610.58501493, 1648.00515436, 1641.46617769])

In [9]:
width_x1

array([1635.70221524, 1660.57553144, 1729.96359604])

In [10]:
pky

array([0.0073849 , 0.00818279, 0.00846554])

In [11]:
pkx[np.where(np.argsort(pky)<2)]

array([1630.4677551 , 1659.66918367])

In [12]:
width_x0[np.where(np.argsort(pky)<2)]

array([1610.58501493, 1648.00515436])

In [13]:
width_x1[np.where(np.argsort(pky)<2)]

array([1982.25273872])